In [12]:
import os
import cv2
import torch
import requests
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from transformers import AutoImageProcessor, AutoModelForKeypointMatching

In [21]:
processor = AutoImageProcessor.from_pretrained("zju-community/efficientloftr")
model = AutoModelForKeypointMatching.from_pretrained("zju-community/efficientloftr")

In [23]:
def find_matches(image1_path, image2_path, score_threshold=0.99):
    # --- Create output directory ---
    os.makedirs("LoFTR_results", exist_ok=True)
    filename = os.path.basename(image1_path).split('.')[0] + "_" + os.path.basename(image2_path).split('.')[0]

    # --- Load images ---
    image1 = Image.open(image1_path).convert("RGB")
    image2 = Image.open(image2_path).convert("RGB")
    images = [image1, image2]
    
    # --- Preprocess and run model ---
    inputs = processor(images=images, return_tensors="pt")
    outputs = model(**inputs)

    # Format sizes as expected: List of [(H, W), (H, W)]
    image_sizes = [(image.height, image.width) for image in images]
    processed_outputs = processor.post_process_keypoint_matching(outputs, [image_sizes], threshold=score_threshold)

    # --- Convert PIL to NumPy ---
    img1 = np.array(image1)
    img2 = np.array(image2)
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]

    # --- Create side-by-side canvas ---
    out_img = np.zeros((max(h1, h2), w1 + w2, 3), dtype=np.uint8)
    out_img[:h1, :w1] = img1
    out_img[:h2, w1:] = img2

    # --- Draw matches ---
    match_count = 0
    for kp0, kp1, score in zip(
        processed_outputs[0]["keypoints0"],
        processed_outputs[0]["keypoints1"],
        processed_outputs[0]["matching_scores"]
    ):
        if score < score_threshold:
            continue
        pt1 = tuple(np.round(kp0.numpy()).astype(int))
        pt2 = tuple(np.round(kp1.numpy()).astype(int) + np.array([w1, 0]))
        color = tuple(np.random.randint(0, 255, size=3).tolist())
        cv2.line(out_img, pt1, pt2, color, 1, cv2.LINE_AA)
        cv2.circle(out_img, pt1, 2, color, -1, cv2.LINE_AA)
        cv2.circle(out_img, pt2, 2, color, -1, cv2.LINE_AA)
        match_count += 1

    # --- Save image ---
    save_path = os.path.join("LoFTR_results", filename + ".png")
    cv2.imwrite(save_path, out_img[:, :, ::-1])
    print(f"[✓] Saved {match_count} high-confidence matches at: {save_path}")

In [ ]:

# --- Path to directory ---
image_dir = "LoFTR_images"

# --- List and sort image filenames ---
image_files = sorted([
    f for f in os.listdir(image_dir)
    if f.lower().endswith((".jpg", ".jpeg", ".png"))
])

# --- Loop through consecutive pairs ---
for i in range(len(image_files) - 1):
    img1_path = os.path.join(image_dir, image_files[i])
    img2_path = os.path.join(image_dir, image_files[i + 1])
    find_matches(img1_path, img2_path)
